Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [1]:
import tensorflow as tf
import os
import zipfile
from os import path, getcwd, chdir

# DO NOT CHANGE THE LINE BELOW. If you are developing in a local
# environment, then grab happy-or-sad.zip from the Coursera Jupyter Notebook
# and place it inside a local folder and edit the path to that location
path = f"{getcwd()}/../tmp2/happy-or-sad.zip"

zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

### Modeling
주의점 : binary classification이니까 modeling 마지막에 softmax말고 sigmoid 쓸거임
* sigmoid는 0~1 사이의 확률값을 반환한다. class 0은 0에 가까운 값을 반환하고, class 1은 1에 가까운 값을 반환한다.
* 따라서 마지막 layer의 neuron은 1개여야 함!
* Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')


binary classification이니까 loss도 binary_crossentropy 사용. optimizer는 Adam 말고 RMSprop도 써 보자.
* RMSprop 특징 : Learning rate(`lr` 파라미터)를 조정해가면서 optimize 가능
* In this case, using the RMSprop optimization algorithm is preferable to stochastic gradient descent (SGD), because RMSprop automates learning-rate tuning for us. (Other optimizers, such as Adam and Adagrad, also automatically adapt the learning rate during training, and would work equally well here.)

In [7]:
# GRADED FUNCTION: train_happy_sad_model
def train_happy_sad_model():
    # Please write your code only where you are indicated.
    # please do not remove # model fitting inline comments.

    DESIRED_ACC = 0.999

    class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('acc')>DESIRED_ACC):
                print("\n\nI did it!!")
                self.model.stop_training = True

    callbacks = myCallback()
    
    # This Code Block should Define and Compile the Model. Please assume the images are 150 X 150 in your implementation.
    model = tf.keras.models.Sequential([
        # 이제 convolutional layer 끼얹기
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
        tf.keras.layers.MaxPooling2D(2,2),

        tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),

        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),

        # 일단 DNN 구조먼저
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid') #binary classification
        # Your Code Here
    ])

    from tensorflow.keras.optimizers import RMSprop

    # model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001), metrics=['accuracy'])
    model.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['accuracy'])
        

    # This code block should create an instance of an ImageDataGenerator called train_datagen 
    # And a train_generator by calling train_datagen.flow_from_directory

    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    train_datagen = ImageDataGenerator(rescale=1./255)

    # Please use a target_size of 150 X 150.
    train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s',
        target_size=(150,150),
        batch_size=8,
        class_mode='binary')  # binary_crossentropy loss를 사용하므로 binary labels 필요
    # Expected output: 'Found 80 images belonging to 2 classes'

    # This code block should call model.fit_generator and train for
    # a number of epochs.
    # model fitting
    history = model.fit_generator(
        train_generator,
        steps_per_epoch=10,
        epochs=20, # 20번 돌리고 0.999 이상 나오면 멈추도록
        verbose=1,
        callbacks=[callbacks])
    # model fitting
    return history.history['acc'][-1]

In [ ]:
# The Expected output: "Reached 99.9% accuracy so cancelling training!""
train_happy_sad_model()

커널을 종료하고 memory resource 해제하기

In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)